In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Play default agents
To get a sense of how the game works, 4 default agents are set to play against each other: 2 greedy agents and 2 random agents.

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese")
env.reset()
env.run([random_agent, random_agent, greedy_agent, greedy_agent])
env.render(mode="ipython", width=800, height=700)

# Simple Agent: Improved Greedy with 2 stages 

Similar to the code for the default Greedy agent, this agent chooses from moves that don't go adjacent to any heads and don't go into any bodies. It then prioritizes the moves as follows:

1. Early game: lay low and maximize distance to the nearest opponent (to avoid getting killed early). Only seek out food greedily when smaller than 3 (avoid starving).

2. Late game: greedily go for food to gain a higher reward (in case many geese survive, the fattest goose wins). Late game is decided based on an estimate of how many time steps are needed to catch up to the longest opponent and limited to a reasonable survivable length of 35.

In [ ]:
%%writefile laylow.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, adjacent_positions, translate, min_distance
import numpy as np
from random import choice

def min_distance_opponent(position, opponents, columns): # same as builtin min_distance_food() but for opponent heads instead of food
    row, column = row_col(position, columns)
    return min(abs(row - opp_r) + abs(column - opp_c) for opponent in opponents for opponent_head in [opponent[0]] for opp_r, opp_c in [row_col(opponent_head, columns)])

class LayLowAgent:
    def __init__(self, configuration: Configuration):
        self.configuration = configuration
        self.last_action = None
        self.stepsleft = 200 # keep track of how many steps left in game to change to greedy late game

    def __call__(self, observation: Observation):
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent in opponents
            for opponent_head in [opponent[0]]
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        # Don't move into any bodies
        bodies = {position for goose in geese for position in goose}
        
        position = geese[observation.index][0] 

        self.stepsleft -= 1
        longestOpp = max([len(opponent) for opponent in opponents])
        aggressCrit = min(max(longestOpp-len(geese[observation.index]),0)*12,40) # define a time step for changing to greedy based on how much longer longest oppenent is
        if len(geese[observation.index]) < 3 or self.stepsleft < aggressCrit and len(geese[observation.index]) < 35: 
            # Move to the closest food
            actions = {
                action: min_distance(new_position, food, columns)
                for action in Action
                for new_position in [translate(position, action, columns, rows)]
                if (
                    new_position not in head_adjacent_positions and
                    new_position not in bodies and
                    (self.last_action is None or action != self.last_action.opposite())
                )
            }
            action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
            self.last_action = action
            return action.name
        else:
            # stay away from enemies
            actions = {
                action: min_distance_opponent(new_position, opponents, columns)
                for action in Action
                for new_position in [translate(position, action, columns, rows)]
                if (
                    new_position not in head_adjacent_positions and
                    new_position not in bodies and
                    (self.last_action is None or action != self.last_action.opposite())
                )
            }
            action = max(actions, key=actions.get) if any(actions) else choice([action for action in Action]) # choose action based on farthest position from closest enemy
            self.last_action = action
            return action.name


cached_agents = {}

def agent(obs, config):
    index = obs["index"]
    if index not in cached_agents:
        cached_agents[index] = LayLowAgent(Configuration(config))
    return cached_agents[index](Observation(obs))

# Debug game 
Confirm the agent works

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese", debug=True)
env.reset()
env.run([greedy_agent, 'laylow.py'])
env.render(mode="ipython", width=500, height=400)

# Evaluate Performance
Evaluate the percentage of games won when two agents play against each other using the get_win_percentages() function from this https://www.kaggle.com/pierretihon/so-far-so-goose-simple-model-based-on-points
<br> Compares with the default greedy agent and itself too to get an idea of performance over 200 iterations.

In [ ]:
import numpy as np

def get_win_percentages(agent1, agent2, n_rounds=200):
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("hungry_geese", [agent1, agent2], num_episodes=n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("hungry_geese", [agent2, agent1], num_episodes= n_rounds-n_rounds//2)]
    p1_wins = [True for [a,b] in outcomes if a>b]
    p2_wins = [True for [a,b] in outcomes if b>a]
    print("Agent 1 Win Percentage:", np.round(len(p1_wins)/len(outcomes), 6)*100)
    print("Agent 2 Win Percentage:", np.round(len(p2_wins)/len(outcomes), 6)*100)
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))
    print(outcomes)

In [ ]:
get_win_percentages(greedy_agent, greedy_agent)

In [ ]:
get_win_percentages('laylow.py', 'laylow.py')

In [ ]:
get_win_percentages(greedy_agent, 'laylow.py')